<a href="https://colab.research.google.com/github/johnJay1745/CVX-AURA-bribing-and-locking/blob/main/Emission_calcs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install duneanalytics

In [ ]:
from duneanalytics import DuneAnalytics
import pandas as pd 
import numpy as np 
import numpy as np
import time
import datetime
from datetime import datetime
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt 
import csv
import json
import requests

In [ ]:
def get_dune_data(id):
    dune = DuneAnalytics('karlkautsky1938', 'karlkautsky1938')
    dune.login()
    dune.fetch_auth_token()
    result_id = dune.query_result_id(query_id = id)
    return dune.query_result(result_id)

In [ ]:
def get_token_supply(token):
    total_supply = (requests.get(
    "https://api.coingecko.com/api/v3/coins/" + 
    token).json()
    ["market_data"]
    ["total_supply"])
    return total_supply

In [ ]:
def get_token_price(token):
    price_usd = (requests.get(
    "https://api.coingecko.com/api/v3/coins/" + 
    token).json()
    ["market_data"]
    ["current_price"]
    ["usd"])
    return price_usd

In [ ]:
tokens = {

    "CVX":{
        "max_supply": 100000000,
        "total_supply":float(get_token_supply("convex-finance")) ,
        "cliff_count": 1000,
        "сliff_size": 100000,
        "current_cliff": 95591144 / 100000,
        "weekly_take_rate": 0.9
    },

    "CRV": {
        "yearly_emission": 194323750.2,
        "community_ption": 0.57,
        "CVX_portion": 0.5
    }

}

In [ ]:
def calc_CVX_emission_array(tokens,growth_multiple):
    CVX_emissions = []
    CVX_total_supplies = []
    df = pd.DataFrame()
    weeks = list(range(0,35))
    CRV_capture_rate = 0.47
    CVX_total_supply = tokens["CVX"]["total_supply"]
    current_cliff = CVX_total_supply / tokens["CVX"]["сliff_size"]
    weekly_CVX_emission = ((
    tokens["CRV"]["yearly_emission"] * 
    7 / 
    365) * 
    CRV_capture_rate)

    cliff_count = tokens["CVX"]["cliff_count"]

    for i in range(len(weeks)):

        remaining = cliff_count - current_cliff
        CVX_emission = weekly_CVX_emission * remaining / cliff_count

        CVX_emissions.append(CVX_emission)
        CRV_capture_rate *= 1.005
        CVX_total_supply += CVX_emission
        CVX_total_supplies.append(CVX_total_supply)
        current_cliff = CVX_total_supply / tokens["CVX"]["сliff_size"]

    df["week"] = weeks
    df["CVX_total_supply"] = CVX_total_supplies
    df["CVX_emissions"] = CVX_emissions
    df["CVX_supply_growth"] = (
    df["CVX_total_supply"].
    shift(1) * 
    growth_multiple / 
    df["CVX_total_supply"])
    
    return df

In [ ]:
def calc_amount_to_lock(tokens,growth_multiple,APR,pool_size,price):
    df = calc_CVX_emission_array(tokens,growth_multiple)
    df["CVX_locked"] = (
    get_dune_data(476828)
    ["data"]
    ["get_result_by_result_id"]
    [0]
    ["data"]
    ["cvx_locked"])
    CVX_required = (APR * 7 / 365) * pool_size / price 
    df["СVX_portion"] = CVX_required / df["CVX_emissions"]
    df["CVX_to_lock"] =  (
    df["CVX_locked"] *
    df["СVX_portion"] *
    price *
    df["CVX_supply_growth"])
    
    return df

In [ ]:
def calc_bribes(tokens,growth_multiple,profit_multiplier,APR,pool_size,price):
    df = calc_amount_to_lock(tokens,profit_multiplier,APR,pool_size,price)
    USD_per_vlCVX_array = []

    vlCVX_delegated_perc = (
    get_dune_data(440612)
    ["data"]
    ["get_result_by_result_id"]
    [0]
    ["data"]
    ["delegated_perc"]) 
    
    df["vlCVX_delegated_on_Votium"] = ((
    get_dune_data(440403)
    ["data"]
    ["get_result_by_result_id"]
    [0]["data"]
    ["delegated_vlcvx"]) * 
    df["CVX_supply_growth"] * 
    vlCVX_delegated_perc / 
    100)

    df["vlCVX_bribe_perc"] = (
    df["СVX_portion"] * 
    df["CVX_locked"] / 
    df["vlCVX_delegated_on_Votium"])
   
    for i in range(len(
        get_dune_data(420765)
        ["data"]
        ["get_result_by_result_id"])):
        
        USD_per_vlCVX = ((
        get_dune_data(420765)
        ["data"]
        ["get_result_by_result_id"]
        [i]
        ["data"]
        ["usd_bribe_per_vlcvx"]) / 
        10)
        USD_per_vlCVX_array.append(USD_per_vlCVX)

    USD_per_vlCVX = np.median(USD_per_vlCVX_array) * profit_multiplier
    df["bribe_amount"] = (
    USD_per_vlCVX  * 
    df["vlCVX_delegated_on_Votium"] * 
    df["vlCVX_bribe_perc"])

    return df

In [ ]:
def get_AURA_stats(BAL_capture_rate, growth_multiplier):
    df = pd.DataFrame()
    AURA_supplies = []
    AURA_emissions = []
    AURA_current_supply = float(get_token_supply("aura-finance"))
    BAL_captured = 0.24
    BAL_weekly_emission = 145000 
    
    for i in list(range(0,35)):
        aura_emitted = (
        (((500 - 
           (AURA_current_supply - 
            50000000) / 
           100000) * 
          2.5 
          + 700) / 
         500) * 
         BAL_weekly_emission * 
         BAL_captured)
        
        AURA_emissions.append(aura_emitted)
        BAL_captured *= growth_multiplier
        AURA_current_supply += aura_emitted
        AURA_supplies.append(AURA_current_supply)
    
    df["week"] = list(range(0,35))
    df["AURA_emission"] = AURA_emissions
    df["AURA_emission_for_LPs"] = df["AURA_emission"] * 0.5
    df["AURA_supply"] = AURA_supplies
    
    return df

In [ ]:
def amount_of_AURA_to_lock(
    BAL_capture_rate,
    growth_multiplier,
    APR,
    pool_size,
    AURA_price):

    df = get_AURA_stats(BAL_capture_rate,growth_multiplier)
    weekly_return = pool_size * APR * 7 / 365
    df["portion_of_vlAURA_needed"] = (
    weekly_return / 
    (AURA_price * 
     df["AURA_emission_for_LPs"]))
    
    vlAURA_init_supply = (
    get_dune_data(916541)
    ["data"]
    ["get_result_by_result_id"]
    [0]
    ["data"]
    ["aura_staked"])

    df["AURA_supply_gwth_rate"] = (
    df
    ["AURA_supply"].
    shift(-1) / 
    df["AURA_supply"]) 

    df["vlAURA_supply"] = vlAURA_init_supply * df["AURA_supply_gwth_rate"]
    df["vlAURA_to_lock"] = (
    df["portion_of_vlAURA_needed"] * 
    df["vlAURA_supply"] * 
    AURA_price)

    return df

In [ ]:
def get_bribe_amount_for_AURA(
    BAL_capture_rate,
    delegation_ratio,
    premium_multiplier,
    growth_multiplier,
    APR,
    pool_size,
    AURA_price,
    USD_per_AURA):
    
    df = amount_of_AURA_to_lock(
    BAL_capture_rate,
    growth_multiplier,
    APR,
    pool_size,
    AURA_price)

    df["delegated_vlAURA"] = delegation_ratio * df["vlAURA_supply"]           
    df["portion_of_delegated_vlAURA_needed"] = (
    df["portion_of_vlAURA_needed"] / 
    delegation_ratio)

    df["bribe_amount"] = (
    df["portion_of_delegated_vlAURA_needed"] * 
    df["delegated_vlAURA"] * 
    USD_per_AURA * 
    premium_multiplier)
    return df

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(get_bribe_amount_for_AURA(0.24,0.1,1,1,0.2,3000000,5,0.08)["week"],
         get_bribe_amount_for_AURA(0.24,0.1,1,1,0.2,3000000,5,0.08)["bribe_amount"],
        label = "profit premium = 0%")
plt.plot(get_bribe_amount_for_AURA(0.24,0.1,1.01,1,0.2,3000000,5,0.08)["week"],
         get_bribe_amount_for_AURA(0.24,0.1,2,1,0.2,3000000,5,0.08)["bribe_amount"],
         label = "profit premium = 1%")
plt.plot(get_bribe_amount_for_AURA(0.24,0.1,1.05,1,0.2,3000000,5,0.08)["week"],
         get_bribe_amount_for_AURA(0.24,0.1,3,1,0.2,3000000,5,0.08)["bribe_amount"],
         label = "profit premium = 5%")
plt.title("weekly vAURA bribes to support 20% APR in $1M pool")
plt.xlabel("week")
plt.ylabel("bribe amount, $")
plt.legend(loc = "upper left")

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(calc_amount_to_lock(tokens,0.8,0.2,1000000,5)["week"],
        calc_amount_to_lock(tokens,0.8,0.2,1000000,5)["CVX_to_lock"],
        label = "growth multiple = 0.8")
plt.plot(calc_amount_to_lock(tokens,1,0.2,1000000,5)["week"],
        calc_amount_to_lock(tokens,1,0.2,1000000,5)["CVX_to_lock"],
        label = "growth multiple = 1")
plt.plot(calc_amount_to_lock(tokens,1.2,0.2,1000000,5)["week"],
        calc_amount_to_lock(tokens,1.2,0.2,1000000,5)["CVX_to_lock"],
        label = "growth multiple = 1.2")

plt.xlabel("week", fontsize = 10)
plt.ylabel("CVX amount to lock, $", fontsize = 10)
plt.legend(loc = "upper left")
plt.title ("vlCVX required to achieve 20% APR in $1M pool")

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(amount_of_AURA_to_lock(0.24,1.001,0.2,3000000,get_token_price("aura-finance"))["week"],
         amount_of_AURA_to_lock(0.24,1.001,0.2,3000000,get_token_price("aura-finance"))["vlAURA_to_lock"],
         label = "growth_multiplier = 1.001"
        )
plt.plot(amount_of_AURA_to_lock(0.24,1.005,0.2,1000000,get_token_price("aura-finance"))["week"],
         amount_of_AURA_to_lock(0.24,1.005,0.2,1000000,get_token_price("aura-finance"))["vlAURA_to_lock"],
         label = "growth multiplier = 1.005"
        )
plt.plot(amount_of_AURA_to_lock(0.24,1.01,0.2,1000000,get_token_price("aura-finance"))["week"],
         amount_of_AURA_to_lock(0.24,1.01,0.2,1000000,get_token_price("aura-finance"))["vlAURA_to_lock"],
         label = "growth multiplier = 1.01"
        )
plt.xlabel("week", fontsize = 10)
plt.ylabel("AURA amount to lock, $", fontsize = 10)
plt.legend(loc = "lower left")
plt.title ("vlAURA required for 30% APR in $3M pool")

In [ ]:
plt.figure(figsize = (8,4))
plt.plot(calc_bribes(tokens,1,1,0.2,1000000,5)["week"],
         calc_bribes(tokens,1,1,0.2,1000000,5)["bribe_amount"],
         label = "profit premium = 0%"
        )
plt.plot(calc_bribes(tokens,1,1.01,0.2,1000000,5)["week"],
         calc_bribes(tokens,1,1.01,0.2,1000000,5)["bribe_amount"],
         label = "profit premium = 1%"
        )
plt.plot(calc_bribes(tokens,1,1.05,0.2,1000000,5)["week"],
         calc_bribes(tokens,1,1.05,0.2,1000000,5)["bribe_amount"],
         label = "profit premium = 5%"
        )
plt.title("weekly vlCVX bribes to support 20% APR in $1M pool")
plt.xlabel("week")
plt.ylabel("bribe amount, $")
plt.legend(loc = "upper left")